## **Vincenty's formula**

 is a method for calculating the distance between two points on an ellipsoidal Earth model. It takes into account the Earth's ellipsoidal shape and provides a more accurate distance calculation compared to simpler methods like the Haversine formula. The formula involves iteratively solving a set of trigonometric equations to find the shortest path, also known as the geodesic, between the two points. This geodesic distance considers variations in both latitude and longitude, making it suitable for longer distances and higher accuracy requirements.



*   mathematical aproach
*   loading Geodesic for the same task



## **mathematical aproach**

In [1]:
import math

# WGS 84 parameters
a = 6378137  # Equatorial radius in meters
f = 1 / 298.257223563  # Flattening factor
b = (1 - f) * a  # Polar radius in meters

MILES_PER_KILOMETER = 0.621371

MAX_ITERATIONS = 1000  # Increased number of iterations
CONVERGENCE_THRESHOLD = 1e-15  # Adjusted convergence threshold

def vincenty_distance(point1, point2, miles=False):
    # Convert latitude and longitude to radians
    lat1, lon1 = math.radians(point1[0]), math.radians(point1[1])
    lat2, lon2 = math.radians(point2[0]), math.radians(point2[1])

    # Calculate U1 and U2
    U1 = math.atan((1 - f) * math.tan(lat1))
    U2 = math.atan((1 - f) * math.tan(lat2))
    L = lon2 - lon1

    # Calculate sin and cos values
    sinU1, cosU1 = math.sin(U1), math.cos(U1)
    sinU2, cosU2 = math.sin(U2), math.cos(U2)

    Lambda = L

    # Iteratively solve for Lambda
    for _ in range(MAX_ITERATIONS):
        sinLambda, cosLambda = math.sin(Lambda), math.cos(Lambda)
        sinSigma = math.sqrt((cosU2 * sinLambda) ** 2 +
                             (cosU1 * sinU2 - sinU1 * cosU2 * cosLambda) ** 2)
        if sinSigma == 0:
            return 0.0  # Coincident points
        cosSigma = sinU1 * sinU2 + cosU1 * cosU2 * cosLambda
        sigma = math.atan2(sinSigma, cosSigma)
        sinAlpha = cosU1 * cosU2 * sinLambda / sinSigma
        cosSqAlpha = 1 - sinAlpha ** 2
        cos2SigmaM = cosSigma - 2 * sinU1 * sinU2 / cosSqAlpha
        C = f / 16 * cosSqAlpha * (4 + f * (4 - 3 * cosSqAlpha))
        LambdaPrev = Lambda
        Lambda = L + (1 - C) * f * sinAlpha * (sigma + C * sinSigma *
                                               (cos2SigmaM + C * cosSigma *
                                                (-1 + 2 * cos2SigmaM ** 2)))
        if abs(Lambda - LambdaPrev) < CONVERGENCE_THRESHOLD:
            break  # Successful convergence
    else:
        return None  # Failure to converge

    # Calculate intermediate values
    uSq = cosSqAlpha * (a ** 2 - b ** 2) / (b ** 2)
    A = 1 + uSq / 16384 * (4096 + uSq * (-768 + uSq * (320 - 175 * uSq)))
    B = uSq / 1024 * (256 + uSq * (-128 + uSq * (74 - 47 * uSq)))
    deltaSigma = B * sinSigma * (cos2SigmaM + B / 4 * (cosSigma *
                 (-1 + 2 * cos2SigmaM ** 2) - B / 6 * cos2SigmaM *
                 (-3 + 4 * sinSigma ** 2) * (-3 + 4 * cos2SigmaM ** 2)))
    s = b * A * (sigma - deltaSigma)

    s /= 1000  # Meters to kilometers
    if miles:
        s *= MILES_PER_KILOMETER  # Kilometers to miles

    return round(s, 6)

# Test the function
point1 = (34.0522, -118.2437)  # Los Angeles
point2 = (37.7749, -122.4194)  # San Francisco

distance_km = vincenty_distance(point1, point2)
distance_miles = vincenty_distance(point1, point2, miles=True)

print(f"Distance in kilometers: {distance_km:.6f} km")
print(f"Distance in miles: {distance_miles:.6f} miles")


Distance in kilometers: 559.042337 km
Distance in miles: 347.372696 miles


## **loading Geodesic for the same task**

In [3]:
from geographiclib.geodesic import Geodesic

# Test the function
point1 = (34.0522, - 118.2437)  # Los Angeles
point2 = (37.7749, -122.4194)  # San Francisco

geod = Geodesic.WGS84
result = geod.Inverse(point1[0], point1[1], point2[0], point2[1])

distance_km = result['s12'] / 1000  # Convert meters to kilometers
distance_miles = distance_km * MILES_PER_KILOMETER  # Convert kilometers to miles

print(f"Distance in kilometers: {distance_km:.6f} km")
print(f"Distance in miles: {distance_miles:.6f} miles")


Distance in kilometers: 559.042337 km
Distance in miles: 347.372696 miles
